In [2]:
import pandas as pd 
from nltk import ngrams
import numpy as np

## Read the data from the files :

### Pre-Processed And Annotated YouTube Data :

In [38]:
data = pd.read_csv('../Data/PreProcessedYouTubeDataFileAndAnnotated.csv')

In [39]:
#data = data[["text","offensive/non offensive"]]
data = data.iloc[:,1:-1]
data.head()

,author,authorChannelUrl,text,likeCount,publishedAt,offensive/non offensive
0,Mokran Tarak,http://www.youtube.com/channel/UCles9OFN0Ngktk...,فمك كبير من وجهك تقبلي في كميرة وجهك صحيحة,60,2020-12-24T21:27:56Z,p
1,Hicham Absaoui,http://www.youtube.com/channel/UCzuY7uYl2regXl...,سلام الله عليكم ورحمته وبركاته قلولهاةتزول الك...,0,2021-04-23T05:12:41Z,n
2,Miss Loona,http://www.youtube.com/channel/UCTnw4ZGfNylgSJ...,افهمي حاجة النداء يكون بصفة الجمع لي مش معني ر...,1,2021-04-22T11:40:22Z,n
3,Amani Maryola,http://www.youtube.com/channel/UCBkuU6BHLiKCfN...,مش تقصد رجال اي تقصد خوك باباك يعني كل رجال ال...,1,2021-04-22T09:37:04Z,n
4,Miss Loona,http://www.youtube.com/channel/UCTnw4ZGfNylgSJ...,لي غاضتو معناها راهي عليه,2,2021-04-22T09:23:21Z,n


### Dictionary with new bad words :
The new bad words have extracted from the offensive comments that have lenght >= 4.

In [40]:
dictionary = pd.read_csv('../Data/dictionary_with_new_bad_words.csv')

Convert the dictionary from DataFram type to Dict type

In [41]:
dictionary = dict(zip(dictionary["bad_word"], dictionary["frequency"]))

### Annotate :
It is a function used to automaticly annotate a comment as offensive if it contains a bad word present in the dictionary 

In [42]:
def annotate(comment):
    dictionary = pd.read_csv('../Data/dictionary_with_new_bad_words.csv')
    dictionary = dict(zip(dictionary["bad_word"], dictionary["frequency"]))
    annotation = "n"
    for n in [1,2,3,4]:
        commentNgrams = ngrams(comment.split(), n)
        for grams in commentNgrams:
            key = ' '.join(grams)
            if key in dictionary : 
                annotation = "p"
    return annotation

In [43]:
data['auto_annotation'] = data["text"].apply(lambda x: annotate(x)) 


In [44]:
data['result'] = np.where(data["offensive/non offensive"] == data["auto_annotation"], 1, 0) 


In [45]:
data.head()

,author,authorChannelUrl,text,likeCount,publishedAt,offensive/non offensive,auto_annotation,result
0,Mokran Tarak,http://www.youtube.com/channel/UCles9OFN0Ngktk...,فمك كبير من وجهك تقبلي في كميرة وجهك صحيحة,60,2020-12-24T21:27:56Z,p,n,0
1,Hicham Absaoui,http://www.youtube.com/channel/UCzuY7uYl2regXl...,سلام الله عليكم ورحمته وبركاته قلولهاةتزول الك...,0,2021-04-23T05:12:41Z,n,n,1
2,Miss Loona,http://www.youtube.com/channel/UCTnw4ZGfNylgSJ...,افهمي حاجة النداء يكون بصفة الجمع لي مش معني ر...,1,2021-04-22T11:40:22Z,n,n,1
3,Amani Maryola,http://www.youtube.com/channel/UCBkuU6BHLiKCfN...,مش تقصد رجال اي تقصد خوك باباك يعني كل رجال ال...,1,2021-04-22T09:37:04Z,n,n,1
4,Miss Loona,http://www.youtube.com/channel/UCTnw4ZGfNylgSJ...,لي غاضتو معناها راهي عليه,2,2021-04-22T09:23:21Z,n,n,1


In [46]:
true_annotation = np.sum(data["result"])
print(true_annotation)

1815


In [47]:
false_annotation = len(data) - true_annotation
print(false_annotation)

765


In [48]:
true_annotation*100/ len(data)

70.34883720930233

# Using Annotation :
Using Annotation Function to get offensive Comments from the non annotated data

In [49]:
data_non_annotated = pd.read_csv('../Data/PreProcessedYouTubeDataFileAndNonAnnotated.csv')

In [50]:
data_non_annotated = data_non_annotated.iloc[:,1:-1]

data_non_annotated['auto_annotation'] = data_non_annotated["text"].apply(lambda x: annotate(x)) 
data_non_annotated = data_non_annotated.iloc[:,1:]
data_non_annotated.head()

,authorChannelUrl,text,likeCount,publishedAt,offensive/non offensive,auto_annotation
0,http://www.youtube.com/channel/UCQ9IzqkTPyKz1J...,والله يا مونيا لي راكي تهدري عليهم هوما لي طلع...,0,2020-10-21T14:21:38Z,NaN,n
1,http://www.youtube.com/channel/UCfksSGorX_DMCr...,ماعندو مادخلك فينا حنا خاوة بنات دراري,0,2020-10-21T14:12:51Z,NaN,n
2,http://www.youtube.com/channel/UCMkdl3UbXl0ZjM...,اسما يعني انتي ماشي بنادم,0,2020-10-21T14:07:27Z,NaN,n
3,http://www.youtube.com/channel/UCp4Lr2iRd9USZt...,3 9 3 9 3كثر فيه الحسد النميمة القذف,0,2020-10-21T14:00:58Z,NaN,n
4,http://www.youtube.com/channel/UCUpIDbOPplowFB...,متحكلي متخليني طريق معوج لدبوش بلاد لهريب سعاد...,0,2020-10-21T13:59:50Z,NaN,n


In [51]:
cpt = 0
for index in range(len(data_non_annotated)) :
    comment = data_non_annotated.iloc[index]
    if comment["auto_annotation"] == "p":
        cpt += 1
        comment["offensive/non offensive"] = "p"
        data = data.append(comment)
        #print(comment["text"])
        #print("-------------------")


<ipython-input-51-9c6225a04c8e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comment["offensive/non offensive"] = "p"


In [53]:
print(cpt)
data = data.iloc[:,:7]
data.head()

728


,author,authorChannelUrl,text,likeCount,publishedAt,offensive/non offensive,auto_annotation
0,Mokran Tarak,http://www.youtube.com/channel/UCles9OFN0Ngktk...,فمك كبير من وجهك تقبلي في كميرة وجهك صحيحة,60,2020-12-24T21:27:56Z,p,n
1,Hicham Absaoui,http://www.youtube.com/channel/UCzuY7uYl2regXl...,سلام الله عليكم ورحمته وبركاته قلولهاةتزول الك...,0,2021-04-23T05:12:41Z,n,n
2,Miss Loona,http://www.youtube.com/channel/UCTnw4ZGfNylgSJ...,افهمي حاجة النداء يكون بصفة الجمع لي مش معني ر...,1,2021-04-22T11:40:22Z,n,n
3,Amani Maryola,http://www.youtube.com/channel/UCBkuU6BHLiKCfN...,مش تقصد رجال اي تقصد خوك باباك يعني كل رجال ال...,1,2021-04-22T09:37:04Z,n,n
4,Miss Loona,http://www.youtube.com/channel/UCTnw4ZGfNylgSJ...,لي غاضتو معناها راهي عليه,2,2021-04-22T09:23:21Z,n,n


In [54]:
data['result'] = np.where(data["offensive/non offensive"] == data["auto_annotation"], 1, 0) 
data.head()

,author,authorChannelUrl,text,likeCount,publishedAt,offensive/non offensive,auto_annotation,result
0,Mokran Tarak,http://www.youtube.com/channel/UCles9OFN0Ngktk...,فمك كبير من وجهك تقبلي في كميرة وجهك صحيحة,60,2020-12-24T21:27:56Z,p,n,0
1,Hicham Absaoui,http://www.youtube.com/channel/UCzuY7uYl2regXl...,سلام الله عليكم ورحمته وبركاته قلولهاةتزول الك...,0,2021-04-23T05:12:41Z,n,n,1
2,Miss Loona,http://www.youtube.com/channel/UCTnw4ZGfNylgSJ...,افهمي حاجة النداء يكون بصفة الجمع لي مش معني ر...,1,2021-04-22T11:40:22Z,n,n,1
3,Amani Maryola,http://www.youtube.com/channel/UCBkuU6BHLiKCfN...,مش تقصد رجال اي تقصد خوك باباك يعني كل رجال ال...,1,2021-04-22T09:37:04Z,n,n,1
4,Miss Loona,http://www.youtube.com/channel/UCTnw4ZGfNylgSJ...,لي غاضتو معناها راهي عليه,2,2021-04-22T09:23:21Z,n,n,1


In [59]:
len(data)

3308

In [55]:
true_annotation = np.sum(data["result"])
print(true_annotation)

2543


In [56]:
false_annotation = len(data) - true_annotation
print(false_annotation)

765


In [57]:
true_annotation*100/ len(data)

76.87424425634825

In [37]:
len(data_non_annotated)
data_non_annotated.head()


,text,likeCount,publishedAt,offensive/non offensive,Algerian Dialect,auto_annotation
0,والله يا مونيا لي راكي تهدري عليهم هوما لي طلع...,0,2020-10-21T14:21:38Z,NaN,NaN,n
1,ماعندو مادخلك فينا حنا خاوة بنات دراري,0,2020-10-21T14:12:51Z,NaN,NaN,n
2,اسما يعني انتي ماشي بنادم,0,2020-10-21T14:07:27Z,NaN,NaN,n
3,3 9 3 9 3كثر فيه الحسد النميمة القذف,0,2020-10-21T14:00:58Z,NaN,NaN,n
4,متحكلي متخليني طريق معوج لدبوش بلاد لهريب سعاد...,0,2020-10-21T13:59:50Z,NaN,NaN,n


In [61]:
data.to_csv('../Data/PreProcessedYouTubeDataFileManAndAutoAnnotated.csv')


In [62]:
data.head()

,author,authorChannelUrl,text,likeCount,publishedAt,offensive/non offensive,auto_annotation,result
0,Mokran Tarak,http://www.youtube.com/channel/UCles9OFN0Ngktk...,فمك كبير من وجهك تقبلي في كميرة وجهك صحيحة,60,2020-12-24T21:27:56Z,p,n,0
1,Hicham Absaoui,http://www.youtube.com/channel/UCzuY7uYl2regXl...,سلام الله عليكم ورحمته وبركاته قلولهاةتزول الك...,0,2021-04-23T05:12:41Z,n,n,1
2,Miss Loona,http://www.youtube.com/channel/UCTnw4ZGfNylgSJ...,افهمي حاجة النداء يكون بصفة الجمع لي مش معني ر...,1,2021-04-22T11:40:22Z,n,n,1
3,Amani Maryola,http://www.youtube.com/channel/UCBkuU6BHLiKCfN...,مش تقصد رجال اي تقصد خوك باباك يعني كل رجال ال...,1,2021-04-22T09:37:04Z,n,n,1
4,Miss Loona,http://www.youtube.com/channel/UCTnw4ZGfNylgSJ...,لي غاضتو معناها راهي عليه,2,2021-04-22T09:23:21Z,n,n,1
